In [1]:
from pathlib import Path
import re
import fitz

In [ ]:
path = Path().resolve()
ifrs_path = path / "data" / "raw" / "IFRS_9.pdf"

doc = fitz.open(ifrs_path)

In [5]:
ifrs = ""
for page in doc:
    blocks = page.get_text("blocks")  
    #As the layout can be divided by two columns when there is subparts
    #It's not taken into account normally because it thinks it's the left column and then right one
    blocks = sorted(blocks, key=lambda b: (b[1], b[0])) 
    
    for b in blocks :
        ifrs += b[4]+"\n"

    #The cons of this method is that it follow exactly the layout, so each paragraph will have multiple lines instead of one

In [6]:
ifrs = "Chapter 1 Objective"+re.split("Chapter 1 Objective", ifrs)[1]

#Numerotation
ifrs = re.sub("\nIFRS 9\n","", ifrs)
ifrs = re.sub(".*© IFRS Foundation.*","",ifrs)
ifrs = re.sub(r"\nA\d+\n", "",ifrs)

ifrs = re.split("\nAppendix C\n",ifrs)[0] #Unnecessary appendix

In [7]:
ifrs = re.sub(r"(\w+)\n(\w+)",r"\1 \2",ifrs)
ifrs = re.sub(r"(,)\n(\w+)",r"\1 \2",ifrs)
ifrs = re.sub(r"\n(\w+) \n",r"\1 ",ifrs)
ifrs = re.sub(r"\n\n\n([a-z])",r" \1",ifrs) #If new page and lower than assume sentence not finished
ifrs = re.sub(r"\n\n\n",r"\n",ifrs) #New page have 3 \n

In [8]:
#Optimisation regarding retrieval
ifrs = re.sub(r"\n(\(\w+\))\n",r'\n\1 ',ifrs)

#Highlight parts
ifrs = re.sub(r"\n([1-9]\.[1-9]) ", r'\n##\1 ',ifrs)
ifrs = re.sub(r"\n([1-9]\.[1-9])\n\n", r'\n##\1 ',ifrs)
ifrs = re.sub(r".(\n[1-9]\.[1-9])", r"\n\1", ifrs) #Some part aren't separated enough

Problem : right now table aren't retrieved exactly

In [14]:
save_path = path / "data" / "cleaned" / "IFRS_9.txt"
save_path.write_text(ifrs, encoding="utf-8")

468798